In [2]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.5.1


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))



train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64


In [4]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 13, 13, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
Dense (Dense)                (None, 10)                13530     
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5246 - sparse_categorical_accuracy: 0.8191
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3818 - sparse_categorical_accuracy: 0.8655
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3497 - sparse_categorical_accuracy: 0.8766
Epoch 4/5
1875/1875 [=

In [5]:
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)


export_path = /var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/1

INFO:tensorflow:Assets written to: /var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/1/assets


INFO:tensorflow:Assets written to: /var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/1/assets


In [6]:
!ls -l {export_path}

total 176
drwxr-xr-x  2 bhuwankarki  staff     64 Sep  6 10:31 assets
-rw-r--r--  1 bhuwankarki  staff   7981 Sep  6 10:31 keras_metadata.pb
-rw-r--r--  1 bhuwankarki  staff  80588 Sep  6 10:31 saved_model.pb
drwxr-xr-x  4 bhuwankarki  staff    128 Sep  6 10:31 variables


In [8]:
!saved_model_cli show --dir {export_path} 


The given SavedModel contains the following tag-sets:
'serve'


In [9]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['Conv1_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_Conv1_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['Dense'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          Conv1_input:

In [10]:
!saved_model_cli show --dir {export_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [11]:
!saved_model_cli show --dir {export_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['Conv1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_Conv1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['Dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [37]:
!ls {export_path}

assets            keras_metadata.pb saved_model.pb    variables


In [24]:
export_path

'/var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/1'

In [23]:
!saved_model_cli run --dir {export_path} \
    --tag_set serve \
    --signature_def serving_default \
    --input_example "examples=["test_images"]"

Traceback (most recent call last):
  File "/opt/anaconda3/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/tools/saved_model_cli.py", line 1192, in main
    args.func(args)
  File "/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/tools/saved_model_cli.py", line 747, in run
    tensor_key_feed_dict = load_inputs_from_input_arg_string(
  File "/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/tools/saved_model_cli.py", line 660, in load_inputs_from_input_arg_string
    input_examples = preprocess_input_examples_arg_string(input_examples_str)
  File "/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/tools/saved_model_cli.py", line 570, in preprocess_input_examples_arg_string
    input_dict = preprocess_input_exprs_arg_string(input_examples_str)
  File "/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/tools/saved_model_cli.py", line 546, in preprocess_input_exprs

In [26]:
docker run -p 8500:8500 \ 
-p 8501:8501 \
--mount type=bind,source=/var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/1,target=/models/my_model \
  -e MODEL_NAME=my_model \
-t tensorflow/serving










































































































































































^C


In [ ]:
docker run -p 8501:8501 --name tfserving_resnet \                                                                              ─╯
  --mount type=bind,source=/var/folders/gy/5t7qcg1534769yzrg82rygl40000gn/T/,target=/models/resnet \
  -e MODEL_NAME=resnet -t tensorflow/serving

In [40]:
import requests
SERVER_URL = 'http://localhost:8501/v1/models/resnet:predict'

response = requests.get(SERVER_URL)

In [44]:
import json
def make_prediction(instances):
   data = json.dumps({"signature_name": "serving_default", "instances": instances.tolist()})
   headers = {"content-type": "application/json"}
   json_response = requests.post(SERVER_URL, data=data, headers=headers)
   predictions = json.loads(json_response.text)['predictions']
   return predictions

In [45]:
predictions = make_prediction(test_images[0:4])

In [47]:
predictions[0]

[-7.61080122,
 -12.2339478,
 -8.47783566,
 -8.36922073,
 -7.69248486,
 0.460719585,
 -5.97210169,
 1.77949643,
 -1.52014685,
 5.38633585]

# model optimize with tflite

In [48]:
converter=tf.lite.TFLiteConverter.from_saved_model(export_path)

In [50]:
converter.optimizations=[tf.lite.Optimize.DEFAULT]
tflite_model=converter.convert()

In [51]:
with open("/tmp/model.tflite","wb") as f:
    f.write(tflite_model)

# serving tflite model

In [ ]:
docker run -p 8501:8501 
--name tfserving_resnet \                                                                              ─╯
--mount type=bind,source=/tmp/,target=/models \
-e MODEL_NAME=resnet \
-t tensorflow/serving \
--use_tflite_model=True